In [32]:
import numpy as np
import math

cordinate_matrix=np.array([(0,0),(0.5,0),(0.5,0.25),(0,0.25),(0.25,0.125)])

element_connectivity=np.array([(1,2,5),(5,2,3),(5,3,4),(1,5,4),])

dof=2*len(cordinate_matrix)

structure_stiffness=np.zeros((dof,dof))

thickness=float(input('Thickness of element in metres'))
E=float(input("Enter Young's Modulus in GPa"))
nu=float(input("Enter Poisson's ratio"))

for i in range(0,len(element_connectivity)):
    
    fnode=element_connectivity[i][0]
    snode=element_connectivity[i][1]
    tnode=element_connectivity[i][2]
    fnodecordinate=cordinate_matrix[fnode-1][:]
    snodecordinate=cordinate_matrix[snode-1][:]
    tnodecordinate=cordinate_matrix[tnode-1][:]
    association_matrix=[2*fnode-1,2*fnode,2*snode-1,2*snode,2*tnode-1,2*tnode]
    x1=fnodecordinate[0]
    y1=fnodecordinate[1]
    x2=snodecordinate[0]
    y2=snodecordinate[1]  
    x3=tnodecordinate[0]
    y3=tnodecordinate[1]
    area=(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2))*0.5
  
    b1=y2-y3
    b2=y3-y1
    b3=y1-y2
    a1=x3-x2
    a2=x1-x3
    a3=x2-x1
    B_matrix = np.array([(b1,0, b2, 0,b3,0),(0,a1,0,a2,0,a3),(a1,b1,a2,b2,a3,b3)]) 
    B_matrix=(0.5/area)*B_matrix
    transpose_B_matrix=np.transpose(B_matrix)
    D=np.array([(1,nu,0),(nu,1,0),(0,0,(1-nu)*0.5)])
    D=E*(10**9)/(1-nu**2)*D
    local_element_stiffness=area*thickness*transpose_B_matrix@D@B_matrix
    print('local_element_stiffness  \n',local_element_stiffness)
    print('\n')
    expanded_element_stiffness=np.zeros((dof,dof))
    for j in range(0,6):
        for k in range (0,6):
             expanded_element_stiffness[association_matrix[j]-1,association_matrix[k]-1]=local_element_stiffness[j,k]
    expanded_element_stiffness=np.round(expanded_element_stiffness,3)         
    print('expanded_element_stiffness  \n',expanded_element_stiffness)
    print('\n')         
    structure_stiffness=structure_stiffness+expanded_element_stiffness

print('structure stiffness matrix: \n', structure_stiffness)
print('\n')

force=np.zeros(dof)
displacement=np.zeros(dof)
force[2]=9375
force[4]=9375
force1=np.delete(force, [0, 1,6,7], 0)
print(force,force1)
stiffness=np.delete(structure_stiffness, [0, 1,6,7], 0)
stiffness=np.delete(stiffness, [0, 1,6,7], 1)
print("stiffness \n",stiffness)
print('\n') 
inv_stiffness=np.linalg.inv(stiffness)
displacement1=inv_stiffness@force1
displacement[2]=displacement1[0]
displacement[3]=displacement1[1]
displacement[4]=displacement1[2]
displacement[5]=displacement1[3]
displacement[8]=displacement1[4]
displacement[9]=displacement1[5]

force=structure_stiffness@displacement
force=np.round(force,4)

print('Reaction in Newton and Displacement in metre:\n')
print(('Force Vector \n',force))
print('\n') 
print(('Displacement Vector\n',displacement))
print('\n')
                  
print('rx1=',(force[0])) 
print('ry1=',(force[1]))  
print('rx4=',(force[6]))
print('ry4=',(force[7]))
print('\n')
print('displacement of 2nd node in (x,y)=(',displacement[2],',',displacement[3],')')                   
print('displacement of 3nd node in (x,y)=(',displacement[4],',',displacement[5],')')   
print('\n')                            
stress=np.zeros(3)
u=np.zeros(6)
for i in range(0,len(element_connectivity)):
    fnode=element_connectivity[i][0]
    snode=element_connectivity[i][1]
    tnode=element_connectivity[i][2]
    fnodecordinate=cordinate_matrix[fnode-1][:]
    snodecordinate=cordinate_matrix[snode-1][:]
    tnodecordinate=cordinate_matrix[tnode-1][:]
    association_matrix=[2*fnode-1,2*fnode,2*snode-1,2*snode,2*tnode-1,2*tnode]
    x1=fnodecordinate[0]
    y1=fnodecordinate[1]
    x2=snodecordinate[0]
    y2=snodecordinate[1]  
    x3=tnodecordinate[0]
    y3=tnodecordinate[1]
    area=(x1*(y2-y3)+x2*(y3-y1)+x3*(y1-y2))*0.5
    b1=y2-y3
    b2=y3-y1
    b3=y1-y2
    a1=x2-x3
    a2=x3-x1
    a3=x1-x3
    B_matrix = np.array([(b1,0, b2, 0,b3,0),(0,a1,0,a2,0,a3),(a1,b1,a2,b2,a3,b3)]) 
    B_matrix=(0.5/area)*B_matrix
   
    D=np.array([(1,nu,0),(nu,1,0),(0,0,(1-nu)*0.5)])
    D=E*(10**9)/(1-nu**2)*D
    
    
    for j in range(0,6):
        u[j]=displacement[association_matrix[j]-1]
    #print(u)
    stress=D@B_matrix@u
    print('Stress(N/m^2) in',i,'th element \n',stress)
    print('\n')
    C = (stress[0] + stress[1])/2
    R = ((stress[0] - stress[1])/2)**2 + (stress[2])**2
    M = 2*stress[2]/(stress[0] - stress[1])
    s1=C+(R)**0.5
    s2=C-(R)**0.5
    pi=math.pi
    theta = math.atan((M)/2)*180/pi
    print(' Principal stress(N/m^2) and thetap in',i,'th element \n',s1,s2,theta)
    print('\n')


Thickness of element in metres0.025
Enter Young's Modulus in GPa210
Enter Poisson's ratio0.3
local_element_stiffness  
 [[ 1.73076923e+09  9.37500000e+08  2.88461538e+08 -7.21153846e+07
  -2.01923077e+09 -8.65384615e+08]
 [ 9.37500000e+08  3.13701923e+09  7.21153846e+07  2.63221154e+09
  -1.00961538e+09 -5.76923077e+09]
 [ 2.88461538e+08  7.21153846e+07  1.73076923e+09 -9.37500000e+08
  -2.01923077e+09  8.65384615e+08]
 [-7.21153846e+07  2.63221154e+09 -9.37500000e+08  3.13701923e+09
   1.00961538e+09 -5.76923077e+09]
 [-2.01923077e+09 -1.00961538e+09 -2.01923077e+09  1.00961538e+09
   4.03846154e+09  0.00000000e+00]
 [-8.65384615e+08 -5.76923077e+09  8.65384615e+08 -5.76923077e+09
   0.00000000e+00  1.15384615e+10]]


expanded_element_stiffness  
 [[ 1.73076923e+09  9.37500000e+08  2.88461538e+08 -7.21153846e+07
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.01923077e+09 -8.65384615e+08]
 [ 9.37500000e+08  3.13701923e+09  7.21153846e+07  2.63221154e+09
   0.000